# **RAGAgent: An Internet-Augmented Conversational Research Assistant**

## **Project Overview**

**RAGAgent** is an intelligent agentic research assistant that performs **web-based Retrieval-Augmented Generation (RAG)** using open-source large language models. It integrates live internet search, semantic chunk retrieval, document embeddings, and tool-augmented reasoning to answer complex questions with grounded, real-world information.

This project demonstrates how local LLMs — specifically, **Mistral-7B-Instruct** — can be combined with the LangChain framework to simulate a powerful multi-turn assistant capable of retrieving and synthesizing external knowledge dynamically.

> **All components run locally** — without reliance on OpenAI APIs — making this setup **cost-effective, private, and fully customizable**.
>
> **Note on Output Visibility**
>
> Some output cells have been intentionally cleared to ensure compatibility across platforms like **Kaggle** and **GitHub**.
>
> This primarily includes cells related to:
>  - Large model initialization (e.g., Mistral-7B)
>  - Embedding generation
>  - Chroma vectorstore setup
>  - Agent responses with multi-turn dialogue
>
> All code executes correctly, and full outputs can be reproduced in local or Colab environments.


## **Objectives**

- Use DuckDuckGo to retrieve relevant web URLs based on user queries.
- Parse and split online documents into text chunks for granular search.
- Encode text chunks using MiniLM embeddings and store them in a Chroma vector database.
- Register a custom tool for semantic search over vectorized web content.
- Enable agentic, multi-turn dialogue using Mistral-7B-Instruct with memory.
- Produce answers that are context-aware and grounded in real-world sources.

## **Core Components**

- **Web Search**: `duckduckgo_search.DDGS` for retrieving query-relevant URLs  
- **Document Loader**: `WebBaseLoader` to fetch and clean webpage content  
- **Text Chunking**: `RecursiveCharacterTextSplitter` for chunk-level processing  
- **Embeddings**: `all-MiniLM-L6-v2` via `HuggingFaceEmbeddings`  
- **Vectorstore**: `Chroma` for persistent semantic retrieval  
- **LLM**: `mistralai/Mistral-7B-Instruct-v0.1`, loaded locally using `transformers.pipeline`  
- **Tool**: LangChain `Tool()` wrapper around the retriever  
- **Agent**: `ConversationalReactDescription` agent with tool use and memory  
- **Memory**: `ConversationBufferMemory` to maintain context across dialogue turns  

#### **Environment Setup & Requirements Installation**

Installs dependencies and prepares the environment by downloading necessary files and packages for training and evaluation.

In [ ]:
%pip install langchain duckduckgo-search transformers accelerate bitsandbytes trafilatura faiss-cpu sentence-transformers langchain-community -U langchain-huggingface bitsandbytes

In [ ]:
!pip install chromadb

#### **Import Required Libraries**

Import core libraries and modules required for model loading, web search, document retrieval, embedding, vector storage, agent creation, memory handling, and Hugging Face integration.

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from duckduckgo_search import DDGS
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.agents import create_react_agent, AgentExecutor, Tool
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.messages import SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.tools.render import render_text_description_and_args
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from huggingface_hub import login

#### **Hugging Face Authentication**

Authenticates with the Hugging Face Hub to access pre-trained models and embedding repositories.

In [ ]:
login()

#### **Model and Pipeline Initialization**

Loads the `Mistral-7B-Instruct-v0.1` language model and tokenizer locally using the Transformers library, then wraps the model in a Hugging Face generation pipeline for downstream use in LangChain agents.

> Model loaded successfully — output cleared for compatibility across Kaggle/GitHub

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=pipe)

#### **Web Search Query and URL Retrieval**

Executes a DuckDuckGo web search using the user-defined query and extracts the top five URLs for downstream document retrieval and processing.

In [6]:
query = "Recent advancements in edge AI and their applications"
results = []

with DDGS() as ddgs:
    for r in ddgs.text(query, max_results=5):
        results.append(r["href"])

urls = results
print("🔗 Fetched URLs:")
print(urls)

🔗 Fetched URLs:
['https://www.sciencedirect.com/science/article/pii/S2667345223000196', 'https://link.springer.com/article/10.1007/s10586-024-04686-y', 'https://arxiv.org/pdf/2407.04053', 'https://www.wevolver.com/article/2023-edge-ai-technology-report', 'https://ieeexplore.ieee.org/document/9453402']


#### **Web Document Loading and Flattening**

Loads the contents of the retrieved URLs using `WebBaseLoader` and flattens the list of documents into a single structured list for subsequent chunking and vectorization.

In [7]:
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [doc for sublist in docs for doc in sublist]

#### **Document Chunking**

Uses `RecursiveCharacterTextSplitter` to divide loaded documents into smaller overlapping text chunks. This prepares the data for efficient embedding and semantic retrieval.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=30)
chunks = text_splitter.split_documents(docs_list)

#### **Embedding and Vectorstore Construction**

Initializes the `all-MiniLM-L6-v2` model using `HuggingFaceEmbeddings` to transform document chunks into vector representations. These embeddings are then stored in a persistent `Chroma` vectorstore under the `chroma_db` directory for efficient semantic retrieval.

> Vectorstore created and persisted — output cleared to reduce cell clutter

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    documents=chunks,
    collection_name="agentic-rag-chroma",
    embedding=embedding_model,
    persist_directory="chroma_db"
)

retriever = vectorstore.as_retriever()

#### **Tool Definition**

Defines a custom tool named `search_context` using LangChain’s `Tool()` interface. The tool allows the agent to retrieve semantically relevant text chunks based on a user’s question. 

In [10]:
tool = Tool(
    name="search_context",
    func=retriever.invoke,
    description="Searches the web content and returns relevant chunks based on the user's question.",
)

tools = [tool]

#### **Memory Initialization**

Creates a `ConversationBufferMemory` instance to maintain chat history and context across multiple agent turns, enabling coherent follow-up interactions.

In [11]:
memory = ConversationBufferMemory(memory_key="chat_history")

<ipython-input-11-1d4038f3f304>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


#### **Agent Initialization**

Initializes a `ConversationalReactDescription` agent using the LLM, registered tools, and memory. The agent is configured for verbose output to support tool-augmented multi-turn dialogue.

In [12]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

<ipython-input-12-23aac195fbc1>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


#### **Initial Agent Invocation**

Executes an agent query to demonstrate tool-augmented reasoning and retrieval-based response generation for a complex information-seeking task.

In [13]:
response = agent.invoke({"input": "What are some real-world use cases of edge AI in 2025?"})



> Entering new AgentExecutor chain...
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool t

#### **Follow-Up Queries and Interaction**

Executes follow-up prompts that request clarification, simplification, and deeper elaboration. Demonstrates the agent’s memory-driven conversation capabilities and iterative knowledge refinement.

> Agent invoked — output was cleared to avoid rendering issues on GitHub/Kaggle

In [ ]:
agent.invoke({"input": "Can you repeat the last answer in simpler terms?"})

In [16]:
response = agent.invoke({"input": "Can you elaborate on its use in autonomous vehicles?"})



> Entering new AgentExecutor chain...
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool t

In [17]:
print(response['output'])

Edge AI is a critical component of autonomous vehicles, as it allows for real-time decision making based on data from sensors and cameras. This is important because autonomous vehicles operate in dynamic environments, and they must be able to adapt to changing conditions and make decisions quickly in order to stay safe on the road.

Edge AI systems in autonomous vehicles typically consist of a combination of sensors, such as cameras, lidars, and radars, and a central processing unit (CPU) that processes the data from these sensors in real-time. The CPU is responsible for performing tasks such as object detection, lane keeping, and path planning, and it uses edge AI algorithms to make decisions about how to control the vehicle's movements based on the data it receives from the sensors.

One of the key advantages of using edge AI in autonomous vehicles is that it allows for faster decision making. By processing data locally, rather than sending it to a centralized server for analysis, ed

In [18]:
response = agent.invoke({"input": "What are the primary challenges and limitations of integrating LLMs into safety-critical systems like autonomous vehicles, and what solutions are being explored to address them?"})



> Entering new AgentExecutor chain...
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool t

In [19]:
print(response['output'])

Edge AI is a critical component of autonomous vehicles, as it allows for real-time decision making based on data from sensors and cameras. This is important because autonomous vehicles operate in dynamic environments, and they must be able to adapt to changing conditions and make decisions quickly in order to stay safe on the road.

Edge AI systems in autonomous vehicles typically consist of a combination of sensors, such as cameras, lidars, and radars, and a central processing unit (CPU) that processes the data from these sensors in real-time. The CPU is responsible for performing tasks such as object detection, lane keeping, and path planning, and it uses edge AI algorithms to make decisions about how to control the vehicle's movements based on the data it receives from the sensors.

One of the key advantages of using edge AI in autonomous vehicles is that it allows for faster decision making. By processing data locally, rather than sending it to a centralized server for analysis, ed

## **Final Thoughts**

**RAGAgent** illustrates how modern Retrieval-Augmented Generation systems can be constructed entirely with open-source tools to deliver intelligent, context-aware responses grounded in real-time web content.

By integrating a local LLM (`Mistral-7B-Instruct`) with semantic retrieval, agentic reasoning, and persistent memory, the system effectively simulates a self-contained research assistant capable of multi-turn dialogue. Its modular architecture — powered by LangChain and Hugging Face — provides a scalable foundation for building more specialized or domain-specific agents.

---

**Thank you for exploring RAGAgent**.